In [1]:
from enum import Enum   

class System(Enum):
    PI3 = 1
    PI4 = 2
    FDP = 3

system_in_use = System.PI3

Data Verification

In [87]:
import clr

clr.AddReference('System')
from System import String, Double
from System.Collections.Generic import Dictionary

import os, sys

backend_cs_dll = '/nfs/home/brendan.posehn/ws/table_verification/table_verification_cs/bin/Debug/' # get some better way to do this 
sys.path.append(backend_cs_dll)
clr.AddReference('table_verification_cs')

from TableVerification import TableVerifier, FailingProfile

fp = FailingProfile()
fp.ProfileColumnName = "B161087"
fp.SearchAxis = "NevinsA"

tbl_params = Dictionary[String, Double]()
tbl_params['psieq_soak'] = 0.01
tbl_params['beta_pol1_setpoint'] = 0.1
tbl_params['psieq_dc'] = -0.01
tbl_params['NevinsC'] = 0.26
tbl_params['NevinsN'] = 4
tbl_params['CurrentRatio'] = 0.1
fp.TableParams = tbl_params

fp.OffendingValue = -0.5

offending_vals = [-1, -0.5, 0] # get this from tableaxes later on

print(os.getenv("AURORA_REPOS"))
sys.path.append(os.getenv("AURORA_REPOS"))

from flagships.post_processing.GenerateColumns import get_full_file_path

print(fp.SearchAxis)

for val in offending_vals:
    tbl_params[fp.SearchAxis] = val
    filepath = get_full_file_path("/mnt/lut/Flagships/pi3b/asbuilt/pfc17500ab/2022-06-09/equil", "", ".2022-06-09", soak = tbl_params['psieq_soak'],
            beta=tbl_params['beta_pol1_setpoint'], dc=tbl_params['psieq_dc'], NevinsC=tbl_params['NevinsC'], 
            NevinsAB=tbl_params['NevinsA'], NevinsN=tbl_params['NevinsN'], current_ratio=tbl_params['CurrentRatio'])
    if(not os.path.exists(filepath)):
        filepath = filepath[:filepath.find('ab', filepath.find('equil'))+3] + filepath[filepath.find('ab', filepath.find('equil'))+4:]
    if(not os.path.exists(filepath)):
        filepath = filepath[:filepath.rfind('ab')+3] + filepath[filepath.rfind('ab')+4:]
        print(filepath)
    if(not os.path.exists(filepath)):
        print('FAILED')

# table_name = "pi3b_asbuilt_pfc17500ab_2022-06-09"
# v = TableVerifier(table_name, "B161087,B211100,B261087,B291060,B215008,B261008")

# bad_profiles = str(tv.checkProfileAtColumns("bbb", 11.111))

# print("\nhere are bad profiles: ", bad_profiles)


/home/brendan.posehn@gf.local/dev/gf
NevinsA
/mnt/lut/Flagships/pi3b/asbuilt/pfc17500ab/2022-06-09/equil\s000_bp0.10_dc+000\ab-1.0_c0.26_n04\s000_bp0.10_dc+000_ab-1.0_c0.26_n04_i00.10.2022-06-09.hdf5
FAILED
/mnt/lut/Flagships/pi3b/asbuilt/pfc17500ab/2022-06-09/equil\s000_bp0.10_dc+000\ab-0.5_c0.26_n04\s000_bp0.10_dc+000_ab-0.5_c0.26_n04_i00.10.2022-06-09.hdf5
FAILED
/mnt/lut/Flagships/pi3b/asbuilt/pfc17500ab/2022-06-09/equil\s000_bp0.10_dc+000\ab+0.0_c0.26_n04\s000_bp0.10_dc+000_ab+0.0_c0.26_n04_i00.10.2022-06-09.hdf5
FAILED


1. Check for holes in the data

In [88]:
# s="/mnt/lut/Flagships/pi3b/asbuilt/pfc17500ab/2022-06-09/equil/s000_bp0.10_dc+000/ab-01.0_c0.26_n04/s000_bp0.10_dc+000_ab-01.0_c0.26_n04_i00.10.2022-06-09.hdf5"

s = "/mnt/lut/Flagships/pi3b/asbuilt/pfc17500ab/2022-06-09/equil/s000_bp0.10_dc+000/ab+0.0_c0.26_n04/s000_bp0.10_dc+000_ab+0.0_c0.26_n04_i00.10.2022-06-09.hdf5"

# if(not os.path.exists(s)):
#     s = s[:s.find('ab', s.find('equil'))+3] + s[s.find('ab', s.find('equil'))+4:]

# print(s)

# if(not os.path.exists(s)):
#     s = s[:s.rfind('ab')+3] + s[s.rfind('ab')+4:]
#     print(s)

# if(not os.path.exists(s)):
#     print('FAIL')

# print(s[s.find('ab', s.find('equil')):])


In [68]:
s = '0123456789'
print(s[:2] + s[3:])

013456789


2. Check for data discontinuities

In [ ]:
import os
import sys

print(os.getenv("AURORA_REPOS"))
sys.path.append(os.getenv("AURORA_REPOS"))

from flagships.post_processing import GenerateColumns

if(system_in_use == System.PI3):
    csv_generator =  GenerateColumns.PI3CSVGenerator()
elif(system_in_use == System.PI4):
    csv_generator =  GenerateColumns.PI4CSVGenerator()
elif(system_in_use == System.FDP):
    csv_generator = GenerateColumns.FDPCase22a_CSVGenerator()
else:
    print('Ensure correct system is set')

other_cols = ['WBPol', 'q020', 'q050', 'q080', 'WBPolNoDCInFC', 'phiPlInFC']
b_col_names = []
for val in csv_generator.add_bprobe_columns([]):
    b_col_names.append('B' + val[0])

profile_columns_of_interest = other_cols + b_col_names
existent_columns_of_interest = []

all_cols = csv_generator.create_all_columns()
all_col_names = [col.Name for col in all_cols]

for col_of_interest in profile_columns_of_interest:
    if(all_col_names.count(col_of_interest) != 0):
        existent_columns_of_interest.append(col_of_interest)

print(existent_columns_of_interest)




In [ ]:
error_profiles = ""
error_profile_lines = error_profiles.splitlines()

for line in error_profile_lines:
    table_axes_values = line[line.find('(')+1:line.find(')')].split(',')
    values = line.split(',')
    column_name = values[0]
    search_axis = values[1]
